This script takes bounding boxes in the yolo-format and generates masks in the mvtec-ad format. Masks that contain **pedestrians** are considered anomalous.

Yolo: [x_center, y_center, width, height] between [0,1]
mvtec-ad: Black png image with white pixels where ground truth is

In [14]:
%load_ext autoreload
%autoreload 2

import sys
import os.path

sys.path.append("..")

from config import SELECTED_DATASET
from data_loader.data_loader import *

import fiftyone as fo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
dataset_info = load_dataset_info(
    SELECTED_DATASET,
    config_path="/home/dbogdoll/mcity_data_engine/datasets/datasets.yaml",
)
loader_function = dataset_info.get("loader_fct")
dataset = globals()[loader_function](dataset_info)
dataset.persistent = False

 100% |███████████████| 2073/2073 [4.5s elapsed, 0s remaining, 594.5 samples/s]       


INFO:eta.core.utils: 100% |███████████████| 2073/2073 [4.5s elapsed, 0s remaining, 594.5 samples/s]       


 100% |█████████████████| 671/671 [1.2s elapsed, 0s remaining, 564.3 samples/s]         


INFO:eta.core.utils: 100% |█████████████████| 671/671 [1.2s elapsed, 0s remaining, 564.3 samples/s]         


Computing metadata...


INFO:fiftyone.core.metadata:Computing metadata...


 100% |███████████████| 2744/2744 [279.1ms elapsed, 0s remaining, 9.8K samples/s]      


INFO:eta.core.utils: 100% |███████████████| 2744/2744 [279.1ms elapsed, 0s remaining, 9.8K samples/s]      


In [16]:
from PIL import Image, ImageDraw
import numpy as np

target_folder = "/home/dbogdoll/mcity_data_engine/datasets/midadvrb_ano_ped_masks"

anomalous_view = dataset.match_tags("val")
for sample in anomalous_view:
    img_width = sample.metadata.width
    img_height = sample.metadata.height
    filename = os.path.basename(sample.filepath).replace(".jpg", "")
    mask = Image.new("L", (img_width, img_height), 0)
    # Create a black image
    draw = ImageDraw.Draw(mask)
    for bbox in sample.ground_truth.detections:
        if bbox.label == "pedestrian":
            # Convert YOLO format to pixel coordinates
            x_center, y_center, width, height = bbox.bounding_box
            x_center *= img_width
            y_center *= img_height
            width *= img_width
            height *= img_height

            # Calculate the bounding box corners
            x_min = int(x_center - width / 2)
            x_max = int(x_center + width / 2)
            y_min = int(y_center - height / 2)
            y_max = int(y_center + height / 2)

            # Draw the white rectangle for the bounding box
            draw.rectangle([x_min, y_min, x_max, y_max], fill=255)

    # Save the mask
    mask.save(os.path.join(target_folder, f"{filename}.png"))